#Segmentation

In [5]:
#Segment customers by repayment behavior: On-time, Slightly Delayed, Chronic Delayers, Defaulters. What’s the size of each segment?
import pandas as pd
df = pd.read_csv(r"C:/Users/joinr/OneDrive/Documents/vedika_loan_data.csv")


df.assign(Days = pd.to_numeric(df['Days Overdue'], errors='coerce').fillna(0),
          Status = df['Loan Status'].str.lower()) \
  .assign(Segment=lambda x: x.apply(
      lambda row: 'Defaulters' if row['Status'] == 'default'
      else 'On-time' if row['Days'] == 0
      else 'Slightly Delayed' if row['Days'] <= 15
      else 'Chronic Delayers', axis=1)) \
  .Segment.value_counts()


Segment
On-time             26000
Slightly Delayed    12457
Chronic Delayers    11543
Name: count, dtype: int64

In [15]:
#Within age groups (18–30, 31–45, 46–60, 60+), which group has highest default ratio?

df.dropna(subset=['Age', 'Loan Status']) \
  .assign(AgeGroup=pd.cut(df['Age'], bins=[18,30,45,60,120], labels=['18–30','31–45','46–60','60+']),
          Status=df['Loan Status'].str.lower()) \
  .groupby('AgeGroup', observed=True) \
  .apply(lambda x: (x['Status'] == 'default').mean()) \
  .sort_values(ascending=False)




C:\Users\joinr\AppData\Local\Temp\ipykernel_11932\1778213017.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: (x['Status'] == 'default').mean()) \


AgeGroup
18–30    0.0
31–45    0.0
46–60    0.0
60+      0.0
dtype: float64

In [22]:
#What’s the default rate by Employment Type and Education Level?

df.dropna(subset=['Employment Type', 'Education Level', 'Loan Status']) \
  .assign(Status = df['Loan Status'].str.lower()) \
  .assign(Default = lambda x: x['Status'] == 'default') \
  .groupby(['Employment Type', 'Education Level'], observed=True)['Default'] \
  .mean().sort_values(ascending=False)


Employment Type    Education Level
Business Owner     Graduate           0.0
                   Post-Graduate      0.0
                   Primary            0.0
                   Secondary          0.0
Daily Wage Worker  Graduate           0.0
                   Post-Graduate      0.0
                   Primary            0.0
                   Secondary          0.0
Salaried           Graduate           0.0
                   Post-Graduate      0.0
                   Primary            0.0
                   Secondary          0.0
Self-Employed      Graduate           0.0
                   Post-Graduate      0.0
                   Primary            0.0
                   Secondary          0.0
Name: Default, dtype: float64

In [24]:
#Compare loan amounts of first-time vs repeat customers (based on other loans with bank).

df.merge(df.groupby('Customer ID')['Loan ID'].nunique().rename('Loan Count'), on='Customer ID') \
  .assign(CustomerType=lambda x: x['Loan Count'].apply(lambda n: 'First-Time' if n == 1 else 'Repeat')) \
  .groupby('CustomerType')['Loan Amount'].mean().sort_values(ascending=False)


CustomerType
First-Time    35065.35574
Name: Loan Amount, dtype: float64

In [28]:
#Are low loyalty score customers concentrated in specific communication modes?


df.dropna(subset=['Customer Loyalty Score', 'Preferred Communication Mode']) \
  .assign(Score = pd.to_numeric(df['Customer Loyalty Score'], errors='coerce')) \
  .assign(LowFlag = lambda x: x['Score'] < 50) \
  .groupby('Preferred Communication Mode')['LowFlag'].mean().sort_values(ascending=False)


Preferred Communication Mode
App Notification    0.497204
SMS                 0.491142
Phone Call          0.489143
Email               0.482781
Name: LowFlag, dtype: float64